### Deep Reinforcement Learning on SnL


In [21]:
# imports!!

## gym
import gym
from gym import spaces

## rest
import random
import numpy as np
import pickle
from tqdm import tqdm
from snl_gym_environment import SNL_env
import os

In [12]:
from stable_baselines3 import A2C,PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
import os


# Instantiate the env
env = SNL_env()
# check environment validity
check_env(env, warn=True)
log_dir = "./tmp/gym/PPO/"
os.makedirs(log_dir, exist_ok=True)

# Create and wrap the environment
env = SNL_env()
# Logs will be saved in log_dir/monitor.csv
env = Monitor(env, log_dir)
# Model
model = PPO('MlpPolicy', env=env, verbose=1, tensorboard_log="./ppo_snl_tensorboard/")

Using cpu device
Wrapping the env in a DummyVecEnv.


In [13]:
model.learn(total_timesteps=1000000)

2023-03-14 00:11:49.188018: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Logging to ./ppo_snl_tensorboard/PPO_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 18       |
|    ep_rew_mean     | 37.8     |
| time/              |          |
|    fps             | 262      |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 2048     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 18            |
|    ep_rew_mean          | 47            |
| time/                   |               |
|    fps                  | 399           |
|    iterations           | 2             |
|    time_elapsed         | 10            |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 0.00036554786 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.1          |
|    

KeyboardInterrupt: 

In [14]:
model = PPO.load('PPO.zip')

In [20]:
model.observation_space

MultiDiscrete([  6 101 101 101 101 101 101 101 101])

In [16]:
env=SNL_env(printing=True)
obs = env.reset()

p1_wins = 0
p2_wins = 0
ties = 0

print(obs)

for i in range(18000):
    action = model.predict(obs.reshape(1,7))

    # action = np.argmax(action)
    
    obs, rewards, dones, info = env.step(action[0][0])
    
    env.render(mode='console')
    #print(dones)
    if dones:
        info = env.render()
        if(info['p1_won']):
            p1_wins += 1
        if(info['p2_won']):
            p2_wins += 1
        if(info['tie']):
            ties += 1
        obs = env.reset()
        # print('p1_wins',p1_wins)
        # print('p2_wins',p2_wins)
        # print('ties',ties)
        # print("-----------")

print('p1_wins',p1_wins)
print('p2_wins',p2_wins)
print('ties',ties)
print("-----------")

################################
environment state:
die value :6.0
board state :
[[0. 0. 0.]
 [0. 0. 0.]]
player 1 init:  <snl_board_gym.Player object at 0x12acd48b0>
player 2 init:  <snl_board_gym.Player object at 0x160703d60>
################################
[5. 0. 0. 0. 0. 0. 0.]


ValueError: Error: Unexpected observation shape (1, 7) for MultiDiscrete environment, please use (9,) or (n_env, 9) for the observation shape.

In [ ]:
from stable_baselines3.common import results_plotter

# Helper from the library
results_plotter.plot_results([log_dir], 1e5, results_plotter.X_TIMESTEPS, "Custom DQN")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb
def moving_average(values, window):
    """
    Smooth values by doing a moving average
    :param values: (numpy array)
    :param window: (int)
    :return: (numpy array)
    """
    weights = np.repeat(1.0, window) / window
    return np.convolve(values, weights, 'valid')


def plot_results(log_folder, title='Learning Curve'):
    """
    plot the results

    :param log_folder: (str) the save location of the results to plot
    :param title: (str) the title of the task to plot
    """
    x, y = ts2xy(load_results(log_folder), 'timesteps')
    y = moving_average(y, window=50)
    # Truncate x
    x = x[len(x) - len(y):]

    fig = plt.figure(title)
    plt.plot(x, y)
    plt.xlabel('Number of Timesteps')
    plt.ylabel('Rewards')
    plt.title(title + " Smoothed")
    plt.show()


In [ ]:
plot_results('./tmp/gym/')

In [ ]:
plot_results(log_dir)